In [43]:
%run ../../Utils/yp_utils.py

import re

# Initial setup

In [2]:
paper_pmid = 15883361
paper_name = 'panavas_nagy_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [30]:
original_data = pd.read_excel('raw_data/hits.xlsx', sheet_name='Sheet1')

In [31]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 108 x 5


In [32]:
original_data.head()

,Gene*,Replication†,p33‡,Transcription†,Function
0,NaN,NaN,Group 1: Protein biosynthesis,NaN,NaN
1,"MRPL32 § , ¶",162,100,111,Protein biosynthesis
2,"RPL1B § , ¶",192,100,39,Protein biosynthesis
3,"RPL7A § , ¶",192,120,97,Protein biosynthesis
4,"RPS21B § , ¶",33,50,70,Protein biosynthesis


In [33]:
original_data['gene'] = original_data['Gene*'].astype(str)

In [34]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [35]:
original_data['gene'] = original_data['gene'].apply(lambda x: x.replace(',',''))

In [36]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [37]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                             Gene*  \
index_input                                          
0                                              NaN   
6                                              NaN   
15                                             NaN   
21                                             NaN   
27                                             NaN   
40                                            VOS9   
41                                             NaN   
49                                             NaN   
57                                             NaN   
63                                             NaN   
75                    Group 10: RNA transcription    
82           Group 11: DNA remodeling, metabolism    
93                     Group 12: Function unknown    

                                                  Replication†  \
index_input                                                      
0                                                        

In [38]:
original_data = original_data.loc[t,:]

In [44]:
original_data['data'] = original_data['Replication†'].apply(lambda x: re.sub("[^0-9]", "", x))

In [47]:
original_data['data'] = pd.to_numeric(original_data['data'], errors='coerce')

In [48]:
original_data.set_index('orf', inplace=True)

In [49]:
original_data = original_data[['data']].copy()

In [50]:
original_data = original_data.groupby(original_data.index).mean()

In [51]:
original_data.shape

(95, 1)

In [52]:
original_data.head()

,data
orf,
YAL021C,45
YBL006C,25
YBL021C,12
YBL052C,27
YBL093C,9


# Prepare the final dataset

In [53]:
data = original_data.copy()

In [54]:
dataset_ids = [16007]
datasets = datasets.reindex(index=dataset_ids)

In [55]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [56]:
data.head()

dataset_id,16007
data_type,value
orf,
YAL021C,45
YBL006C,25
YBL021C,12
YBL052C,27
YBL093C,9


## Subset to the genes currently in SGD

In [57]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [58]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16007
,data_type,value
gene_id,orf,
19,YAL021C,45
94,YBL006C,25
109,YBL021C,12
140,YBL052C,27
181,YBL093C,9


# Normalize

In [59]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [60]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [61]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      16007          
data_type       value    valuez
gene_id orf                    
19      YAL021C    45  6.044073
94      YBL006C    25  3.357818
109     YBL021C    12  1.611753
140     YBL052C    27  3.626444
181     YBL093C     9  1.208815

# Print out

In [62]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [63]:
from IO.save_data_to_db3 import *

In [64]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 15883361...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.38it/s]

Updating the data_modified_on field...
